# Using Parquet

Here's a quick illustrated guide to using Parquet files in pandas. 

In [1]:
# Make a toy dataset

import pandas as pd
import numpy.random as npr
import random
import string

size = 100000

df = pd.DataFrame({'a': npr.randint(0, 10000, size=size), 
                   'b': [''.join(random.choices(string.ascii_uppercase, k=10))
                         for i in range(size)]} )

In [2]:
df.head()

,a,b
0,2242,FSMZFYOGWX
1,3780,USHEQWJIPT
2,9657,MHNYKXBOLB
3,5155,HSEVQEUYUG
4,4885,SMVHFZPNFC


In [3]:
# We can store "b" as a categorical (analogous to Factors in R). 
# This is helpful, for reasons we'll discuss on Tuesday,
# but categoricals are only preserved by fastparquet. 
df['b'] = pd.Categorical(df['b'])
df.dtypes

a       int64
b    category
dtype: object

In [4]:
# Save
df.to_parquet('/users/nick/desktop/test.parquet', engine='fastparquet')

In [5]:
# Reload
df2 = pd.read_parquet('/users/nick/desktop/test.parquet', engine='fastparquet')
df2.head()

,a,b
0,2242,FSMZFYOGWX
1,3780,USHEQWJIPT
2,9657,MHNYKXBOLB
3,5155,HSEVQEUYUG
4,4885,SMVHFZPNFC


In [6]:
# Note that column b is still a categorical. 
df2.dtypes

a       int64
b    category
dtype: object

You can also easily load subsets of columns:

In [7]:
df2 = pd.read_parquet('/users/nick/desktop/test.parquet', 
                      engine='fastparquet',
                      columns=['b'])
df2.head()

,b
0,FSMZFYOGWX
1,USHEQWJIPT
2,MHNYKXBOLB
3,HSEVQEUYUG
4,SMVHFZPNFC


## Chunking with parquet

Parquet allows chunking, but not quite as easily as you can chunk a csv. 

First, you have to save your file into chunks of a size you want. Parquet always saves rows as chunks, but by default each chunk has 50,000,000 rows, which (if you're chunking) may be too many:

In [8]:
# Save into 50,000 row chunks, 
# so we should get file saved into two chunks. 

df.to_parquet('/users/nick/desktop/test.parquet', 
              engine='fastparquet', 
              row_group_offsets=50000)

In [9]:
# Then we have to read it in using the `fastparquet` 
# library itself (there's no way to do this directly from 
# pandas I'm afraid):

from fastparquet import ParquetFile
pf = ParquetFile('/users/nick/desktop/test.parquet')

# Iterates over row groups
for rg in pf.iter_row_groups():
    print(rg)

          a           b
0      2242  FSMZFYOGWX
1      3780  USHEQWJIPT
2      9657  MHNYKXBOLB
3      5155  HSEVQEUYUG
4      4885  SMVHFZPNFC
...     ...         ...
49995  5880  DDCZRSOYYX
49996  4792  HNTHLENGQH
49997   618  OXMWXAILJU
49998   920  HSZUTZYTTL
49999  7686  BTLUKYOPSG

[50000 rows x 2 columns]
          a           b
0      6669  EAVHNWCJJT
1      9183  AHWBRUTJOP
2      9871  AFNMSRRQSL
3      1759  BVFVHYXNRE
4      9755  BDKCYTCGSN
...     ...         ...
49995   520  VAYOVPABGV
49996  7175  TVFRIQKTPF
49997  9917  LYXJVUZYFC
49998  4495  CLRZXCSYJF
49999   313  VBNJOOIICA

[50000 rows x 2 columns]


In [10]:
# So you can filter and work with each group if you want:
all_rows_w_even_a = []
for rg in pf.iter_row_groups():
    rg = rg.loc[(rg['a'] % 2) == 0]
    all_rows_w_even_a.append(rg)

even_rows = pd.concat(all_rows_w_even_a)
even_rows.sample(10)

,a,b
10088,5746,HAZNSYFSSY
22508,2294,YDDHYHYLUW
45752,7374,QXKXUUSQIW
35231,9102,CONWRQMHFV
45856,836,WSRBKDLGOO
23624,7868,NJUOMMFJMQ
27220,8204,MTBOVIWOTB
30374,8950,UNEHOMUWEX
25098,7224,TCZVDZCNFO
17053,3480,SNCRFQVACY
